# Dogs vs Cats | The quick way

WARNING : Put this file in the /courses/dl1 folder of your Fastai folder.

Original file : https://github.com/fastai/fastai/blob/master/courses/dl1/lesson1.ipynb

## Setup the Fastai library

### 1) The wrong way

In [ ]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# This file contains all the main external libs we'll use
from fastai.imports import *

In [ ]:
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

### 2) The quick way

In [ ]:
from fastai.conv_learner import *

## Setup the path to data

The dataset is available at http://files.fast.ai/data/dogscats.zip. You can download it directly on your server by running the following line in your terminal. wget http://files.fast.ai/data/dogscats.zip. You should put the data in a subdirectory of this notebook's directory, called data/. Note that this data is already available in Crestle and the Paperspace fast.ai template.

In [ ]:
# Put the path to your dogscats folder that corresponds to your installation
PATH = "data/dogscats/"

# Size of images when they are used by GPU
sz=224

## Note : If needed, script to create traning and validation sets based on choose RATIO


(from Welton Rodrigo Torres Nascimento) https://github.com/piegu/mag-fofa/blob/master/scripts/create_sets.sh

In [ ]:
torch.cuda.is_available(), torch.backends.cudnn.enabled

## Check your path

In [ ]:
os.listdir(PATH)

In [ ]:
os.listdir(f'{PATH}valid')

In [ ]:
files = os.listdir(f'{PATH}valid/cats')[:5]
files

In [ ]:
img = plt.imread(f'{PATH}valid/cats/{files[0]}')
plt.imshow(img);

## Easy steps to train a world-class image classifier

### 1) Define your model

Fastai notes : Enable data augmentation, and precompute=True

In [ ]:
# Uncomment the below if you need to reset your precomputed activations
shutil.rmtree(f'{PATH}tmp', ignore_errors=True)

In the lesson1.ipynb, Jeremy Howard uses resnet34.

After completing this exercice with resnet34, test resnet50, resnet101 and resnet152. Check if the validation accuracy changes. 

In [ ]:
# our model
# If you want to know the list of the pretrained models under pytorch : http://pytorch.org/docs/master/torchvision/models.html

arch=resnet34
# arch=resnet50
# arch=resnet101
# arch=resnet152

In [ ]:
# our data transformation
tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.1)

In [ ]:
# CODE HERE of the data object using PATH, the test set, tfms and bs=64 (even if it is the batch default value) (1 line)
# data = ...
# CODE HERE of the learn object using the pretrained model, the data object and precompute=True (1 line)
# learn = ...

In [ ]:
# get all details about layers of your model
learn.summary()

In [ ]:
# get the layers structure of your model
learn.get_layer_groups()

### 2) Find the learning rate

Fastai notes : Use lr_find() to find highest learning rate where loss is still clearly improving

In [ ]:
# CODE HERE of the function to search for the best learning rate (1 line)

In [ ]:
# CODE HERE of the function to plot the loss versus the learning rate (1 line)

### 3) Train quickly the last layer of your model 

Fastai notes : Train last layer from precomputed activations for 1-2 epochs

In [ ]:
# CODE HERE for training the learn model with learning rate 1e-2 and 1 epoch (1 line)
# learn.fit(...)

### 4) Train the last layer of your model with data augmentation and SGDR

Fastai notes : Train last layer with data augmentation (i.e. precompute=False) for 2-3 epochs with cycle_len=1

In [ ]:
# CODE HERE for setting the learn model to precompute=False (1 line)

In [ ]:
# CODE HERE for training the learn model with learning rate 1e-2, 3 cycles and one learning rate restart by epoch (1 line)
# learn.fit(...)

In [ ]:
# CODE HERE for saving your model (the values of the parameters/weights) to 224_lastlayer (1 line)

### 5) Allow the layers of the pretrained model to be trained

Fastai notes : Unfreeze all layers

In [ ]:
# CODE HERE for loading your model (the values of the parameters/weights) from 224_lastlayer (1 line)

In [ ]:
# CODE HERE for unfreezing the first layers of the model (1 line)

### 6) Get one learning rate by group of layers (diferential learning rate)

Fastai notes : Set earlier layers to 3x-10x lower learning rate than next higher layer

In [ ]:
# CODE HERE of the function to search for the best learning rate (1 line)

In [ ]:
# CODE HERE of the function to plot the loss versus the learning rate (1 line)

In [ ]:
# CODE HERE for setting the lr variable to an array of 3 learning rates (1 line)
# lr = ...

In [ ]:
# CODE HERE for training the learn model with the diferential learning rate, 3 cycles, one learning rate restart by epoch
# and double the length of a cycle after each cycle (1 line)
# learn.fit(...)

In [ ]:
# CODE HERE for saving your model (the values of the parameters/weights) to 224_all (1 line)

### 7) Get the validation accuracy by using TTA

In [ ]:
# get the log of the prediction for the validation set using TTA()
# log_preds,y = ...

# For each image in the validation set, the TTA() provides with 5 predictions for each class : we need to get the mean of them
# To get the probabilities, do not forget to take the exp of the logs
# probs = ...

In [ ]:
# Get the accuracy of the validation set
accuracy_np(probs, y)

### 8) Get the predictions on the test set by using TTA

In [ ]:
# It is possible to use TTA on the test set because the test set has been passed to learn
log_preds,y = learn.TTA(is_test=True)
probs = np.mean(np.exp(log_preds), axis=0)
preds = np.argmax(probs, axis=1)

### 9) Get the prediction on a specific image

In [ ]:
# Get the path to a specific image of the test set (for example : the number 16)
files = os.listdir(f'{PATH}/test/')
img = open_image(f'{PATH}/test/{files[15]}')
plt.imshow(img)

In [ ]:
# Get the transformation to be applied on the image and then get the prediction
trn_tfms, val_tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
im = val_tfms(img)
pred = learn.predict_array(im[None])
result = np.argmax(pred, axis=1)
print(f'The class is : {data.classes[result[0]]}')

In [ ]:
# Visualize the image after transformation by tfms_from_model()
print(im.shape)
im_show = np.swapaxes(im,0,2)
im_show = np.swapaxes(im_show,0,1)
print(im_show.shape)
plt.imshow(im_show)


In [ ]:
# Display the scatter diagram of the predictions (probabilities) for the image
# nb_classes : the number of classes
matplotlib.pyplot.scatter(list(range(nb_classes), np.exp(pred))


## Analyzing results | Confusion matrix 

In [ ]:
preds = np.argmax(probs, axis=1)
probs = probs[:,1]

A common way to analyze the result of a classification model is to use a [confusion matrix](http://www.dataschool.io/simple-guide-to-confusion-matrix-terminology/). Scikit-learn has a convenient function we can use for this purpose:

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y, preds)

We can just print out the confusion matrix, or we can show a graphical view (which is mainly useful for dependents with a larger number of categories).

In [ ]:
# Increase the display size of the confusion matrix
matplotlib.rcParams['figure.figsize'] = [30,20]


In [ ]:
plot_confusion_matrix(cm, data.classes)

## Analyzing results: looking at pictures

### 1) Statistics on dataset

In [ ]:
fn = PATH+data.trn_ds.fnames[0]; fn

In [ ]:
img = PIL.Image.open(fn); img

In [ ]:
img.size

In [ ]:
size_d = {k: PIL.Image.open(PATH+k).size for k in data.trn_ds.fnames}

In [ ]:
row_sz, col_sz = zip(*size_d.values())
row_sz = np.array(row_sz)
col_sz = np.array(col_sz)

In [ ]:
row_sz[:5]

In [ ]:
plt.hist(row_sz);

In [ ]:
plt.hist(row_sz[row_sz<1000])

In [ ]:
len(col_sz)

In [ ]:
plt.hist(col_sz);

In [ ]:
plt.hist(col_sz[col_sz<1000])

In [ ]:
len(data.trn_ds),len(data.test_ds)

In [ ]:
len(data.classes),data.classes[:5]

### 2) Visualize the dataset

As well as looking at the overall metrics, it's also a good idea to look at examples of each of:
1. A few correct labels at random
2. A few incorrect labels at random
3. The most correct labels of each class (ie those with highest probability that are correct)
4. The most incorrect labels of each class (ie those with highest probability that are incorrect)
5. The most uncertain labels (ie those with probability closest to 0.5).

In [ ]:
# This is the label for a val data
data.val_y

In [ ]:
# from here we know that 'cats' is label 0 and 'dogs' is label 1.
data.classes

In [ ]:
# this gives prediction for validation set. Predictions are in log scale
log_preds = learn.predict()
log_preds.shape

In [ ]:
log_preds[:10]

In [ ]:
preds = np.argmax(log_preds, axis=1)  # from log probabilities to 0 or 1
probs = np.exp(log_preds[:,1])        # pr(dog)

In [ ]:
def rand_by_mask(mask): return np.random.choice(np.where(mask)[0], 4, replace=False)
def rand_by_correct(is_correct): return rand_by_mask((preds == data.val_y)==is_correct)

In [ ]:
def plot_val_with_title(idxs, title):
    imgs = np.stack([data.val_ds[x][0] for x in idxs])
    title_probs = [probs[x] for x in idxs]
    print(title)
    return plots(data.val_ds.denorm(imgs), rows=1, titles=title_probs)

In [ ]:
def plots(ims, figsize=(12,6), rows=1, titles=None):
    f = plt.figure(figsize=figsize)
    for i in range(len(ims)):
        sp = f.add_subplot(rows, len(ims)//rows, i+1)
        sp.axis('Off')
        if titles is not None: sp.set_title(titles[i], fontsize=16)
        plt.imshow(ims[i])

In [ ]:
def load_img_id(ds, idx): return np.array(PIL.Image.open(PATH+ds.fnames[idx]))

def plot_val_with_title(idxs, title):
    imgs = [load_img_id(data.val_ds,x) for x in idxs]
    title_probs = [probs[x] for x in idxs]
    print(title)
    return plots(imgs, rows=1, titles=title_probs, figsize=(16,8))

In [ ]:
# 1. A few correct labels at random
plot_val_with_title(rand_by_correct(True), "Correctly classified")

In [ ]:
# 2. A few incorrect labels at random
plot_val_with_title(rand_by_correct(False), "Incorrectly classified")

In [ ]:
def most_by_mask(mask, mult):
    idxs = np.where(mask)[0]
    return idxs[np.argsort(mult * probs[idxs])[:4]]

def most_by_correct(y, is_correct): 
    mult = -1 if (y==1)==is_correct else 1
    return most_by_mask(((preds == data.val_y)==is_correct) & (data.val_y == y), mult)

In [ ]:
plot_val_with_title(most_by_correct(0, True), "Most correct cats")

In [ ]:
plot_val_with_title(most_by_correct(1, True), "Most correct dogs")

In [ ]:
plot_val_with_title(most_by_correct(0, False), "Most incorrect cats")

In [ ]:
plot_val_with_title(most_by_correct(1, False), "Most incorrect dogs")

In [ ]:
most_uncertain = np.argsort(np.abs(probs -0.5))[:4]
plot_val_with_title(most_uncertain, "Most uncertain predictions")

## Annex : get automatically the best learning rate number

In [ ]:
def plot_loss_change(sched, sma=1, n_skip=20, y_lim=(-0.01,0.01)):
    """
    Plots rate of change of the loss function.
    Parameters:
        sched - learning rate scheduler, an instance of LR_Finder class.
        sma - number of batches for simple moving average to smooth out the curve.
        n_skip - number of batches to skip on the left.
        y_lim - limits for the y axis.
    """
    derivatives = [0] * (sma + 1)
    for i in range(1 + sma, len(learn.sched.lrs)):
        derivative = (learn.sched.losses[i] - learn.sched.losses[i - sma]) / sma
        derivatives.append(derivative)
        
    plt.ylabel("d/loss")
    plt.xlabel("learning rate (log scale)")
    plt.plot(learn.sched.lrs[n_skip:], derivatives[n_skip:])
    plt.xscale('log')
    plt.ylim(y_lim)

In [ ]:
# search for best learning rate
lrf=learn.lr_find()
#learn.sched.plot_lr()
learn.sched.plot()

In [ ]:
plot_loss_change(learn.sched, sma=20)